In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ ||
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_.
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.




In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we
continue to use the CPU.



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_.
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [13]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)

pred_probab = nn.Softmax(dim=1)(logits)
print(f"A la salida el modelo devuelve un vector de {pred_probab.shape[1]} dimensiones: {pred_probab}")

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

A la salida el modelo devuelve un vector de 10 dimensiones: tensor([[0.1041, 0.0955, 0.0882, 0.1006, 0.0941, 0.1067, 0.1016, 0.0971, 0.1052,
         0.1070]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([9], device='cuda:0')


--------------




Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [14]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3482, -0.2042, -0.3230,  0.4637, -0.0115,  0.5710,  0.0204, -0.0943,
          0.1062,  0.0608,  1.1138, -0.3508, -0.2918,  0.2021,  0.2653, -0.0837,
          0.0419, -0.3878,  0.2909,  0.0568],
        [ 0.3290, -0.0923, -0.2911,  0.0464, -0.0694,  1.1556, -0.1348,  0.4461,
          0.1378, -0.2230,  0.5648, -0.2266, -0.7024, -0.2988,  0.0513, -0.2839,
          0.2602, -0.3804,  0.1914,  0.3349],
        [ 0.0775, -0.1029,  0.0022,  0.3135, -0.5336,  0.6772, -0.1569,  0.3549,
         -0.2259, -0.1742,  1.0651, -0.2274, -0.1081,  0.0755,  0.2639, -0.1319,
          0.1037, -0.3263,  0.4008, -0.0169]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3482, 0.0000, 0.0000, 0.4637, 0.0000, 0.5710, 0.0204, 0.0000, 0.1062,
         0.0608, 1.1138, 0.0000, 0.0000, 0.2021, 0.2653, 0.0000, 0.0419, 0.0000,
         0.2909, 0.0568],
        [0.3290, 0.0000, 0.0000, 0.0464, 0.0000, 1.1556, 0.0000, 0.4461, 0.1378,
         0.0000, 0.5648, 0.0000, 0.0000, 0.0000, 0.051

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

print(logits.shape)

torch.Size([3, 10])


nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [23]:
print(f"Before softmax: {logits}\n\n")

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

print(f"After softmax: {pred_probab}, tamaño softmax: {pred_probab.shape}")

Before softmax: tensor([[-5.7377e-02, -1.2222e-01, -8.2057e-03,  9.5361e-02,  1.8130e-01,
          5.2884e-02, -1.9386e-01, -5.2647e-02, -1.8042e-01,  6.9721e-02],
        [-1.1124e-01, -2.4536e-01, -1.8986e-02,  8.0791e-02,  2.0279e-01,
          1.4338e-01, -2.3234e-01, -1.0678e-01, -1.8027e-01,  8.2203e-02],
        [-1.0899e-01, -2.8749e-01, -1.5164e-02,  4.1711e-02,  1.7841e-01,
          8.0407e-05, -2.3392e-01, -1.8834e-01, -1.8943e-01,  5.4784e-02]],
       grad_fn=<AddmmBackward>)


After softmax: tensor([[0.0958, 0.0898, 0.1007, 0.1116, 0.1217, 0.1070, 0.0836, 0.0963, 0.0847,
         0.1088],
        [0.0919, 0.0804, 0.1008, 0.1114, 0.1258, 0.1186, 0.0814, 0.0923, 0.0858,
         0.1115],
        [0.0957, 0.0800, 0.1051, 0.1112, 0.1275, 0.1067, 0.0844, 0.0884, 0.0883,
         0.1127]], grad_fn=<SoftmaxBackward>), tamaño softmax: torch.Size([3, 10])


Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [24]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0340, -0.0172, -0.0002,  ...,  0.0053,  0.0223,  0.0262],
        [ 0.0355, -0.0075,  0.0030,  ..., -0.0189,  0.0178,  0.0091]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0093, 0.0284], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0079,  0.0209, -0.0043,  ...,  0.0073,  0.0242, -0.0260],
        [-0.0122, -0.0067, -0.0145,  ..., -0.0187, -0.0101,  0.0068]],
       device='cuda:0', grad_fn=<Slic

--------------




Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

